In [1]:
import os

In [2]:
%pwd

'/home/user/Documents/ML PROJECTS/EndToEndRedWineQualityMLProject/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/user/Documents/ML PROJECTS/EndToEndRedWineQualityMLProject'

In [6]:
from dataclasses import dataclass
from RedWineQualityMLProject import logging
from pathlib import Path

In [14]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str


In [8]:
from RedWineQualityMLProject.utils.common  import  create_directories,read_yaml
from RedWineQualityMLProject.constants import *

In [16]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(schema_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories(self.config.artifacts_root)
    def get_modelTraniner_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN
        create_directories(config.root_dir)
        model_trainer_config  = ModelTrainerConfig(root_dir=config.root_dir,train_data_path=config.train_data_path, test_data_path=config.test_data_path,model_name=config.model_name, target_column=schema.name)
        return model_trainer_config

In [17]:
import pandas as pd
import pickle as pkl
from RedWineQualityMLProject import logger
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor


In [26]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        x_train = train_data.drop(self.config.target_column, axis=1)
        x_test = test_data.drop(self.config.target_column, axis=1)
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]
        std_scaler = StandardScaler()
        x_train = std_scaler.fit_transform(x_train)
        x_test = std_scaler.transform(x_test)
        rfr = RandomForestRegressor()
        rfr.fit(x_train,y_train)
        with open(os.path.join(self.config.root_dir[0], self.config.model_name), 'wb') as f:
            pkl.dump(rfr, f)
        
        
        





In [27]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_modelTraniner_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-10-01 12:22:46,025: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-01 12:22:46,032: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-01 12:22:46,036: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-01 12:22:46,037: INFO: common: created directory at: artifacts]
[2023-10-01 12:22:46,038: INFO: common: created directory at: artifacts/model_trainer]
